# Importing/cleaning data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from math import ceil

%matplotlib inline

### Import turnstile data
This cell runs the script we created to easily import multiple weeks worth of data from the MTA website. Running the code will create an mta_data folder in the current directory which contains csv files for all the weeks between the date given as an argument and the most recent Saturday.

In [ ]:
import get_data as gd
gd.main(['2020-12-01'])

### Our primary Python modules
These are the files that we created to streamline creating, cleaning and processing the data. In addition to functions to creating data frames, they also contain functions to create frames aggregated in a variety different ways and functions to add various metrics to the data frame.

*Note: The wrangle_data module assumes that the mta data was imported with the get_data script, and thus looks for data in an mta_data folder in the current directory in the format produced by get_data*

In [1]:
import wrangle_data as wd
import get_metrics as gm

### Create our main data frames for EDA: 
- df - contains the turnstile data, with unique ID's created for each turnstile, booth, and station. Also has added columns for metrics such as net entries and exits, traffic and turnstile count.
    * The wd.run() function takes up to two optional arguments in the form 'YYYY-MM-DD' (Date mut be a Saturday). If no argument is given, it will return a data frame with data from the week ending in 06/27/2020. If one argument is given, will return a data frame with data from the week ending in the Saturday given. If two arguments are given, it will return a data frame with data from all the weeks between that of the first argument given and that of the second.
- df_c - contains the above data frame merged with the data set we acquired with remote complex id information. 
- spt - contains the spatial data set we acquired, which has longitude and latitude coordinates for each station along with more info about the stations themselves. Unfortunately, we were unable to connect the two sets by station with the time we had, as that would require more language processing than we had time for. Therefore, we used the remote complex ID (added to the turnstile data in df_c) to connect the two sets. 
- key - contains a map with all complex IDs and the station IDs and names associated with them.

In [2]:
df = wd.run()
df_c = wd.merge_complex(df)
spt = wd.spt()
key = df_c[['complex_id', 'suid', 'station']].drop_duplicates()

In [4]:
df_c.columns

Index(['datetime', 'c_a', 'unit', 'scp', 'station_x', 'linename', 'division',
       'desc', 'entries', 'exits', 'tuid', 'suid', 'buid', 'ts_count',
       'net_entries', 'net_exits', 'traffic', 'booth', 'complex_id',
       'station_y', 'line_name'],
      dtype='object')

# Defining our basic metrics

This function creates a subset of the complex data frame containing only the most relevant columns. Then, it performs the following processes to further clean the data:
- Creates a delta_time column in order to correct for inconsistent time stamps
- Calculates traffic per hour for each turnstile
- Rounds the datetime column to the frequency given in argument
- For each timestamp, sums the traffic per hour to create a column contaning total traffic per hour of the station
- For each timestamp, divides the total traffic per station by the number of turnstiles in that station in order to  approximate density


In [ ]:
def density_calc(df, freq):
    dens = df_c[['station', 'suid', 'tuid', 'datetime', 'traffic', 'ts_count']]
    
    #Create column with timedelta between current row and next row
    dens['delta_time'] = dens.groupby(['suid', 'tuid'])['datetime'].diff().dt.seconds.shift(-1) / 3600
    #drop the nulls that were created in the above step
    dens = dens.dropna()
    
    #Round each datetime stamp to closest hour to make plotting much more simple
    #Seriously this step reduces unique datetime values from ~11500 to 165
    #The round function is only available for a datetime index, so need to temporarily make 'datetime' the index
    dens = dens.set_index('datetime') 
    dens.index = dens.index.round('H')
    dens.reset_index(inplace=True)
    
    #there are some columns where the delta_time is 0. We don't need these
    dens = dens[dens['delta_time'] > 0]
    
    #Calculate average traffic per hour at each turnstile in each time interval 
    dens['trf_hr'] = dens['traffic'] / dens['delta_time']
    
    #Calculate total traffic per hour per station in each time interval 
    dens_traffic_tot = dens.groupby(['suid', 'datetime'])['trf_hr'].sum().reset_index()
    dens_traffic_tot.rename(columns = {'trf_hr': 'total_traffic'}, inplace=True)
    dens = dens.merge(dens_traffic_tot, on=['suid', 'datetime'], how='left')
    
    #Calculate traffic per turnstile per station in each time interval 
    dens['density'] = dens['total_traffic'] / dens['ts_count']
    
    #Now that we have the station average, return only one row per station w/ relevant columns
    dens = dens[['datetime', 'station', 'suid', 'total_traffic', 'density']].drop_duplicates()
    
    return dens


# Exploratory EDA

Create a dictionary with keys from the unique station IDs and values containing data for all timestamps associated with the corresponding station

In [5]:
dens_station = density_calc(df_c, 'H').set_index('suid')
st_stats = {}
for st in dens_stats_st.index.unique():
    st_stats[dens_stats_st['suid'][st].unique()[0]] = dens_station[dens_station.index == st].reset_index(drop=True)

This function generates and plots an array of subplots

In [ ]:
def sub_arr(df_dict, var, col_nums=10, title='', ylabel='', xlabel=''):
    row_nums = ceil(len(df_dict) / col_nums)  # how many rows of plots
    plt.figure(figsize=(40,100)) 
    
    #create subplots for each station id in dict
    for i, (k, v) in enumerate(df_dict.items(), 1):
        plt.subplot(row_nums, col_nums, i)  
        sub_v = v[var]
        p = plt.stem(sub_v, markerfmt=' ', use_line_collection=True)
        plt.title((title + k))
        plt.ylabel(ylabel)
        plt.xlabel(xlabel)

    plt.tight_layout()
    plt.show()
    return

Running this cell generates a large array of subplots, with each subplot corresponding to a single station and  containing a graph of total traffic vs datetime

In [ ]:
sub_arr(st_stats, 'total_traffic', title='Station ID: ', xlabel='Date/time',
        ylabel='Total traffic/hour in station')

Running this cell generates a large array of subplots, with each subplot corresponding to a single station and containing a graph of density vs datetime

In [ ]:
sub_arr(st_stats, 'density', title='Station ID: ', xlabel='Date/time',
        ylabel='Average density/hour in station')

Creates two dictionaries with keys from each unique timestamp and values containing data from all stations for that specific timestamp. In one dictionary, the stations are sorted by total_traffic, while in the other, the stations are sorted by density.

In [6]:
dens_time = density_calc(df_c, '2H').set_index('datetime')

total_sort = dens_time.reset_index().sort_values(['datetime','total_traffic']).set_index('datetime')
density_sort = dens_time.reset_index().sort_values(['datetime','density']).set_index('datetime')

time_total = {}
for ti in total_sort.index.unique():
    #some timeslots only have data from a few stations so let's ignore those
    if (total_sort[total_sort.index == ti]['suid'].nunique() > 10):
        time_total[ti] = total_sort[total_sort.index == ti].reset_index(drop=True)
        
time_dens = {}
for ti in density_sort.index.unique():
    #some timeslots only have data from a few stations so let's ignore those
    if (density_sort[density_sort.index == ti]['suid'].nunique() > 10):
        time_dens[ti] = density_sort[density_sort.index == ti].reset_index(drop=True)

Running this cell generates a large array of subplots, with each subplot corresponding to a single timestamp and containing a graph of total traffic vs stations, with stations sorted by total traffic

In [ ]:
sub_arr(time_total, 'total_traffic', title='Date/time: ', xlabel='Stations sorted by total traffic',
        ylabel='Total traffic/hour in station', col_nums=5)

Running this cell generates a large array of subplots, with each subplot corresponding to a single timestamp and containing a graph of density vs stations, with stations sorted by total traffic

In [ ]:
sub_arr(time_total, 'density', title='Date/time: ', xlabel='Stations sorted by total traffic',
        ylabel='Density of station', col_nums=5)

Running this cell generates a large array of subplots, with each subplot corresponding to a single timestamp and containing a graph of total traffic vs stations, with stations sorted by density

In [ ]:
sub_arr(time_total, 'total_traffic', title='Date/time: ', xlabel='Stations sorted by density',
        ylabel='Total traffic/hour in station', col_nums=5)

Running this cell generates a large array of subplots, with each subplot corresponding to a single timestamp and containing a graph of density vs stations, with stations sorted by density

In [ ]:
sub_arr(time_total, 'density', title='Date/time: ', xlabel='Stations sorted by density',
        ylabel='Density of station', col_nums=5)

In [6]:
dens_time_wk = density_calc(df_c, '2H').set_index('datetime')
dens_time_wk = dens_time_wk.groupby('suid')[['total_traffic', 'density']].sum()

total_sort_wk = dens_time.sort_values('total_traffic')
density_sort_wk = dens_time.sort_values('density')

Running this cell generates two subplots, with total traffic and density (respectively) plotted by station, with stations sorted by total traffic

In [ ]:
plt.figure(figsize = (30,30))

plt.subplot(2, 1, 1)
p = plt.stem(total_sort_wk['total_traffic'], markerfmt=' ', use_line_collection=True)
plt.title('Total traffic per station', fontsize=35)
plt.ylabel('Total traffic/hour weekly average for ea. station', fontsize = 30)
plt.xlabel('Stations sorted by total weekly traffic', fontsize = 30)

plt.subplot(2, 1, 2)
p = plt.stem(total_sort_wk['density'], markerfmt=' ', use_line_collection=True)
plt.title('Density per station', fontsize=35)
plt.ylabel('Density/hour weekly average for ea. station', fontsize = 30)
plt.xlabel('Stations sorted by total weekly traffic', fontsize = 30)

plt.show()

Running this cell generates two subplots, with total traffic and density (respectively) plotted by station, with stations sorted by density

In [ ]:
plt.figure(figsize = (30,30))

plt.subplot(2, 1, 1)
p = plt.stem(density_sort_wk['total_traffic'], markerfmt=' ', use_line_collection=True)
plt.title('Total traffic per station', fontsize=35)
plt.ylabel('Total traffic/hour weekly average for ea. station', fontsize = 30)
plt.xlabel('Stations sorted by density', fontsize = 30)

plt.subplot(2, 1, 2)
p = plt.stem(density_sort_wk['density'], markerfmt=' ', use_line_collection=True)
plt.title('Density per station', fontsize=35)
plt.ylabel('Density/hour weekly average for ea. station', fontsize = 30)
plt.xlabel('Stations sorted by density', fontsize = 30)

plt.show()

# Primary Analysis

Distribution of ridership over time

Correlation between traffic and density

# Developing a Priority Score

In [ ]:
daily_avg = density_calc(df_c, 'min').set_index('datetime')
daily_avg['date'] = density_calc.index.date
daily_avg = daily_avg.groupby(['suid', 'date'])[['total_traffic', 'density']].sum()

In [ ]:
def priority(df):
    #Calculate priority score
    traf_top = df['traffic'].max()
    dens_top = df['density'].max()
    traffic_weight, density_weight = 0, 0
    tra_score = (df['traffic'] / traf_top) 
    dens_score = (df['density'] / dens_top) 
    df['priority'] = (tra_score + traffic_weight) * (dens_score + density_weight)

    #normalize priority score
    p_min = df_daily_avg['priority'].min()
    p_max = df_daily_avg['priority'].max()
    p_range = p_max - p_min
    df['priority']=(df['priority']-p_min)/(p_range)
    
    return df

In [ ]:
daily_avg = priority(daily_avg)

# Scope
## % of total traffic in top n station by priority order

# Passing Data to CSV for Mapping (by complex ID)

In [ ]:
df_all = wd.run('2020-05-02', '2020-06-27')
df_c_all = wd.merge_complex(df_all)

#aggregate by complex id and date
df_daily_co = wd.agg_by(df_c_all, 'date', 'complex')

#create a turnstile count/complex column
df_cc = df_c.groupby('complex_id')['tuid'].nunique().to_dict()
df_daily_co['ts_count'] = df_daily_co.complex_id.map(df_cc)

#calculate density
df_daily_co['density'] = df_daily_co['traffic'] / df_daily_co['ts_count']

#calculate average traffic and density figures for entire date range
df_daily_co = df_daily_co.groupby('complex_id')[['traffic', 'density']].mean()

#calculate priority scores
df_daily_co = priority(df_daily_co)

In [ ]:
#Export data to csv's so John can implement mapping
df_daily_avg['traffic'].to_csv(path_or_buf='complex_traffic.csv', index=True)
df_daily_avg['density'].to_csv(path_or_buf='complex_density.csv', index=True)
df_daily_avg['priority'].to_csv(path_or_buf='complex_priority.csv', index=True)